In [1]:
from os import listdir
import warnings
warnings.filterwarnings("ignore")
import string
from tqdm import tqdm
import re
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import numpy as np
import seaborn as sns

## Sample story
The summary is the text after @highlight

In [2]:
file = open('cnn/stories' + '/' + '000c835555db62e319854d9f8912061cdca1893e.story', encoding='utf-8')
text = file.read()
file.close()
text

'(CNN)For the second time during his papacy, Pope Francis has announced a new group of bishops and archbishops set to become cardinals -- and they come from all over the world.\n\nPope Francis said Sunday that he would hold a meeting of cardinals on February 14 "during which I will name 15 new Cardinals who, coming from 13 countries from every continent, manifest the indissoluble links between the Church of Rome and the particular Churches present in the world," according to Vatican Radio.\n\nNew cardinals are always important because they set the tone in the church and also elect the next pope, CNN Senior Vatican Analyst John L. Allen said. They are sometimes referred to as the princes of the Catholic Church.\n\nThe new cardinals come from countries such as Ethiopia, New Zealand and Myanmar.\n\n"This is a pope who very much wants to reach out to people on the margins, and you clearly see that in this set," Allen said. "You\'re talking about cardinals from typically overlooked places, 

In [3]:
def loading_articles(file_name):
    file = open(file_name, encoding='utf-8')
    text = file.read()
    file.close()
    return text

In [4]:
def split_story(doc):
    index = doc.find('@highlight')
    story, highlights = doc[:index], doc[index:].split('@highlight')
    highlights = [h.strip() for h in highlights if len(h) > 0]
    return story, highlights

In [5]:
path = 'cnn_stories'
stories = list()
directory='cnn/stories'
for name in tqdm(listdir(directory)):
    filename = directory + '/' + name
    doc = loading_articles(filename)
    story, highlights = split_story(doc)
    stories.append({'story':story, 'highlights':highlights})
print('Loaded Stories %d' % len(stories))

100%|███████████████████████████████████████████████████████████████████████████| 92579/92579 [06:41<00:00, 230.59it/s]

Loaded Stories 92579


In [6]:
cnn_df = pd.DataFrame.from_dict(stories)
cnn_df.columns = ['article','summary']
cnn_df.to_csv('CNN_with_summary.csv',index=False)

In [7]:
CNN=pd.read_csv('CNN_with_summary.csv')
CNN.head(1)

article  \
0  It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria.\n\nObama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons.\n\nThe proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction."\n\nIt's a step that is set to turn an international crisis into a fierce domestic political battle.\n\nThere are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react?\n\nIn a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but because he wants to.\n\n"While I believe I have the authority to carry out this military action without specific congressional authorization, I know that the country will be stronger if we take this course, and our actions will be even more effective," he said. "We should have this debate, because the issues are too big for business as usual."\n\nObama said top congressional leaders had agreed to schedule a debate when the body returns to Washington on September 9. The Senate Foreign Relations Committee will hold a hearing over the matter on Tuesday, Sen. Robert Menendez said.\n\nTranscript: Read Obama's full remarks\n\nSyrian crisis: Latest developments\n\nU.N. inspectors leave Syria\n\nObama's remarks came shortly after U.N. inspectors left Syria, carrying evidence that will determine whether chemical weapons were used in an attack early last week in a Damascus suburb.\n\n"The aim of the game here, the mandate, is very clear -- and that is to ascertain whether chemical weapons were used -- and not by whom," U.N. spokesman Martin Nesirky told reporters on Saturday.\n\nBut who used the weapons in the reported toxic gas attack in a Damascus suburb on August 21 has been a key point of global debate over the Syrian crisis.\n\nTop U.S. officials have said there's no doubt that the Syrian government was behind it, while Syrian officials have denied responsibility and blamed jihadists fighting with the rebels.\n\nBritish and U.S. intelligence reports say the attack involved chemical weapons, but U.N. officials have stressed the importance of waiting for an official report from inspectors.\n\nThe inspectors will share their findings with U.N. Secretary-General Ban Ki-moon Ban, who has said he wants to wait until the U.N. team's final report is completed before presenting it to the U.N. Security Council.\n\nThe Organization for the Prohibition of Chemical Weapons, which nine of the inspectors belong to, said Saturday that it could take up to three weeks to analyze the evidence they collected.\n\n"It needs time to be able to analyze the information and the samples," Nesirky said.\n\nHe noted that Ban has repeatedly said there is no alternative to a political solution to the crisis in Syria, and that "a military solution is not an option."\n\nBergen:  Syria is a problem from hell for the U.S.\n\nObama: 'This menace must be confronted'\n\nObama's senior advisers have debated the next steps to take, and the president's comments Saturday came amid mounting political pressure over the situation in Syria. Some U.S. lawmakers have called for immediate action while others warn of stepping into what could become a quagmire.\n\nSome global leaders have expressed support, but the British Parliament's vote against military action earlier this week was a blow to Obama's hopes of getting strong backing from key NATO allies.\n\nOn Saturday, Obama proposed what he said would be a limited military action against Syrian President Bashar al-Assad. Any military at

In [8]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [9]:
article_text=[]
for i in CNN.article.values:
    tt=re.sub(r'\n',' ', i)
    tt=re.sub(r'>',' ', tt)
    tt=re.sub(r'<',' ', tt)
    tt=re.sub(r'(CNN)',' ', tt)
    tt=re.sub(r'LRB',' ', tt)
    tt=re.sub(r'RRB',' ', tt)
    tt = re.sub(r'[" "]+', " ", tt)
    tt=re.sub(r'-- ',' ', tt)
    tt=re.sub(r"([?!¿])", r" \1 ", tt)
    tt=re.sub(r'-',' ', tt)
    tt=tt.replace('/',' ')
    tt=re.sub(r'\s+', ' ', tt)
    tt=decontracted(tt)
    tt = re.sub('[^A-Za-z0-9.,]+', ' ', tt)
    tt = tt.lower()
    article_text.append(tt)

## Cleaned Story

In [10]:
data_article=pd.DataFrame(article_text,columns=['Article'])
data_article

Article
0      it is official u.s. president barack obama wants lawmakers to weigh in on whether to use military force in syria. obama sent a letter to the heads of the house and senate on saturday night, hours after announcing that he believes military action against syrian targets is the right step to take over the alleged use of chemical weapons. the proposed legislation from obama asks congress to approve the use of military force to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction. it is a step that is set to turn an international crisis into a fierce domestic political battle. there are key questions looming over the debate what did u.n. weapons inspectors find in syria what happens if congress votes no and how will the syrian government react in a televised address from the white house rose garden earlier saturday, the president said he would take his case to congress, not because he has to but because he wants to. while i believe i have the authority to carry out this military action without specific congressional authorization, i know that the country will be stronger if we take this course, and our actions will be even more effective, he said. we should have this debate, because the issues are too big for business as usual. obama said top congressional leaders had agreed to schedule a debate when the body returns to washington on september 9. the senate foreign relations committee will hold a hearing over the matter on tuesday, sen. robert menendez said. transcript read obama is full remarks syrian crisis latest developments u.n. inspectors leave syria obama is remarks came shortly after u.n. inspectors left syria, carrying evidence that will determine whether chemical weapons were used in an attack early last week in a damascus suburb. the aim of the game here, the mandate, is very clear and that is to ascertain whether chemical weapons were used and not by whom, u.n. spokesman martin nesirky told reporters on saturday. but who used the weapons in the reported toxic gas attack in a damascus suburb on august 21 has been a key point of global debate over the syrian crisis. top u.s. officials have said there is no doubt that the syrian government was behind it, while syrian officials have denied responsibility and blamed jihadists fighting with the rebels. british and u.s. intelligence reports say the attack involved chemical weapons, but u.n. officials have stressed the importance of waiting for an official report from inspectors. the inspectors will share their findings with u.n. secretary general ban ki moon ban, who has said he wants to wait until the u.n. team is final report is completed before presenting it to the u.n. security council. the organization for the prohibition of chemical weapons, which nine of the inspectors belong to, said saturday that it could take up to three weeks to analyze the evidence they collected. it needs time to be able to analyze the information and the samples, nesirky said. he noted that ban has repeatedly said there is no alternative to a political solution to the crisis in syria, and that a military solution is not an option. bergen syria is a problem from hell for the u.s. obama this menace must be confronted obama is senior advisers have debated the next steps to take, and the president is comments saturday came amid mounting political pressure over the situation in syria. some u.s. lawmakers have called for immediate action while others warn of stepping into what could become a quagmire. some global leaders have expressed support, but the british parliament is vote against military action earlier this week was a blow to obama is hopes of getting strong backing from key nato allies. on saturday, obama proposed what he said would be a limited military action against syrian president bashar al assad. any military attack would not be open ended or include u.s. ground forces, he said. syria is alleged use of c

In [11]:
summary_text=[]
for i in CNN.summary.values:
    tt=re.sub(r'\n',' ', i)
    tt=re.sub(r'>',' ', tt)
    tt=re.sub(r'<',' ', tt)
    tt=re.sub(r'-',' ', tt)
    tt=re.sub(r'(CNN)',' ', tt)
    tt=re.sub(r'LRB',' ', tt)
    tt=re.sub(r'RRB',' ', tt)

    tt = re.sub(r'[" "]+', " ", tt)
    tt=re.sub(r'-- ',' ', tt)
    tt=tt.replace('/',' ')
    tt=re.sub(r'\s+', ' ', tt)
    tt=decontracted(tt)
    tt = re.sub('[^A-Za-z0-9.]+', ' ', tt)
    tt = tt.lower()
    summary_text.append(tt)

In [12]:
summary_text=np.array(summary_text)
summary_text=summary_text.reshape(-1,1)

In [13]:
data_summ=pd.DataFrame(summary_text,columns=['Summary'])

## Cleaned csv file

In [14]:
data_cleaned=data_article.join(data_summ)
data_cleaned

Article  \
0      it is official u.s. president barack obama wants lawmakers to weigh in on whether to use military force in syria. obama sent a letter to the heads of the house and senate on saturday night, hours after announcing that he believes military action against syrian targets is the right step to take over the alleged use of chemical weapons. the proposed legislation from obama asks congress to approve the use of military force to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction. it is a step that is set to turn an international crisis into a fierce domestic political battle. there are key questions looming over the debate what did u.n. weapons inspectors find in syria what happens if congress votes no and how will the syrian government react in a televised address from the white house rose garden earlier saturday, the president said he would take his case to congress, not because he has to but because he wants to. while i believe i have the authority to carry out this military action without specific congressional authorization, i know that the country will be stronger if we take this course, and our actions will be even more effective, he said. we should have this debate, because the issues are too big for business as usual. obama said top congressional leaders had agreed to schedule a debate when the body returns to washington on september 9. the senate foreign relations committee will hold a hearing over the matter on tuesday, sen. robert menendez said. transcript read obama is full remarks syrian crisis latest developments u.n. inspectors leave syria obama is remarks came shortly after u.n. inspectors left syria, carrying evidence that will determine whether chemical weapons were used in an attack early last week in a damascus suburb. the aim of the game here, the mandate, is very clear and that is to ascertain whether chemical weapons were used and not by whom, u.n. spokesman martin nesirky told reporters on saturday. but who used the weapons in the reported toxic gas attack in a damascus suburb on august 21 has been a key point of global debate over the syrian crisis. top u.s. officials have said there is no doubt that the syrian government was behind it, while syrian officials have denied responsibility and blamed jihadists fighting with the rebels. british and u.s. intelligence reports say the attack involved chemical weapons, but u.n. officials have stressed the importance of waiting for an official report from inspectors. the inspectors will share their findings with u.n. secretary general ban ki moon ban, who has said he wants to wait until the u.n. team is final report is completed before presenting it to the u.n. security council. the organization for the prohibition of chemical weapons, which nine of the inspectors belong to, said saturday that it could take up to three weeks to analyze the evidence they collected. it needs time to be able to analyze the information and the samples, nesirky said. he noted that ban has repeatedly said there is no alternative to a political solution to the crisis in syria, and that a military solution is not an option. bergen syria is a problem from hell for the u.s. obama this menace must be confronted obama is senior advisers have debated the next steps to take, and the president is comments saturday came amid mounting political pressure over the situation in syria. some u.s. lawmakers have called for immediate action while others warn of stepping into what could become a quagmire. some global leaders have expressed support, but the british parliament is vote against military action earlier this week was a blow to obama is hopes of getting strong backing from key nato allies. on saturday, obama proposed what he said would be a limited military action against syrian president bashar al assad. any military attack would not be open ended or include u.s. ground forces, he said. syria is alleged use o